In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
import seaborn as sns
from sklearn.model_selection import cross_val_score

from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from textblob import TextBlob, Word

plt.style.use('fivethirtyeight')
%matplotlib inline

In [2]:
file = './data/sig_train_data.csv'

classifier = pd.read_csv(file, encoding='latin1')

In [3]:
classifier.columns = ['question_num', 'question', 'response', 'maturity',
       'misc_info', 'aup_ref', 'iso_num',
       'iso_desc', 'sig_class']

In [4]:
classifier.head()

question_num                                           question response  \
0          A.1  Is there a risk assessment program that has be...      Yes   
1        A.1.1  A risk assessment, conducted within the last 1...      Yes   
2        A.1.2                                   Risk Governance?      Yes   
3        A.1.3  Range of assets to include: people, processes,...      Yes   
4        A.1.4  Range of threats to include: malicious, natura...      Yes   

   maturity                                          misc_info  \
0       5.0  An enterprise risk assessment is performed ann...   
1       NaN                                                NaN   
2       NaN                                                NaN   
3       NaN                                                NaN   
4       NaN                                                NaN   

                                             aup_ref    iso_num  \
0            A.1 IT & Infrastructure Risk Governance  5.1 6.1.2   
1  A.2 IT & Infrastructure Risk Assessment Life C...        8.2   
2            A.1 IT & Infrastructure Risk Governance        NaN   
3            A.1 IT & Infrastructure Risk Governance        NaN   
4            A.1 IT & Infrastructure Risk Governance      6.1.2   

                                            iso_desc        sig_class  
0  Leadership & Commitment, Information Security ...  Risk Assessment  
1               Information security risk assessment  Risk Assessment  
2                                                NaN  Risk Assessment  
3                                                NaN  Risk Assessment  
4               Information Security Risk Assessment  Risk Assessment

In [5]:
classifier.loc[:, 'sig_class'].value_counts()

Application Security                         215
Operations Management                        120
Business Resiliency                          105
Asset and Information Management             104
Server Security                               96
Physical and Environment                      86
End User Device Security                      76
Access Control                                74
Network Security                              66
Incident Event & Communication Management     45
Risk Assessment                               43
Security Policy                               42
Threat Management                             38
Compliance                                    31
Human Resource Security                       28
Privacy                                       18
Organizational Security                       10
Name: sig_class, dtype: int64

In [6]:
classifier.loc[:, 'sig_class_num'] = classifier.loc[:, 'sig_class'].map({
    'Application Security':1,
    'Operations Management':2,
    'Business Resiliency':3,
    'Asset and Information Management':4,
    'Server Security':5,
    'Physical and Environment':6,
    'End User Device Security':7,
    'Access Control':8,
    'Network Security':9,
    'Incident Event & Communication Management':10,
    'Risk Assessment':11,
    'Security Policy':12,
    'Threat Management':13,
    'Compliance':14,
    'Human Resource Security':15,
    'Privacy':16,
    'Organizational Security':17,
})

In [7]:
classifier.loc[:, 'sig_class_num'].value_counts()

1     215
2     120
3     105
4     104
5      96
6      86
7      76
8      74
9      66
10     45
11     43
12     42
13     38
14     31
15     28
16     18
17     10
Name: sig_class_num, dtype: int64

In [8]:
classifier

question_num                                           question response  \
0             A.1  Is there a risk assessment program that has be...      Yes   
1           A.1.1  A risk assessment, conducted within the last 1...      Yes   
2           A.1.2                                   Risk Governance?      Yes   
3           A.1.3  Range of assets to include: people, processes,...      Yes   
4           A.1.4  Range of threats to include: malicious, natura...      Yes   
5           A.1.5                                      Risk scoping?      Yes   
6           A.1.6                                      Risk context?      Yes   
7           A.1.7                                Risk training plan?      Yes   
8           A.1.8                          Risk evaluation criteria?      Yes   
9           A.1.9           Risk scenarios? If yes, do they include:      Yes   
10        A.1.9.1  Events and possible threats that could impact ...      Yes   
11        A.1.9.2                                      Threat types?      Yes   
12         A.1.10  Ownership, action plan, response plan, managem...      Yes   
13            A.2  Is there a program to manage the treatment of ...      Yes   
14          A.2.1  A formal process for assigning appropriate man...      Yes   
15          A.2.2  A formal process for appropriate management kn...      Yes   
16          A.2.3  A formal process for tracking the status of ac...      Yes   
17          A.2.4        Controls identified for each material risk?      Yes   
18          A.2.5  Measures for defining, monitoring, and reporti...      Yes   
19        A.2.5.1                            Performance benchmarks?      Yes   
20        A.2.5.2                Service Level Agreement Compliance?      Yes   
21        A.2.5.3                                 Policy Compliance?      Yes   
22        A.2.5.4                             Control Effectiveness?      Yes   
23        A.2.5.5                          Vulnerability Management?      Yes   
24            A.3  Do Subcontractors have access to Scoped System...       No   
25            A.4              Is there a vendor management program?      Yes   
26          A.4.1  Does the vendor management program include an ...       No   
27        A.4.1.1  Risk rating of the issue (e.g., H/M/L, 1-5, et...       No   
28        A.4.1.2                                 Remediation plans?       No   
29        A.4.1.3                                  Remediation date?       No   
...           ...                                                ...      ...   
1167    U.1.37.15       Password change requirements at first logon?      n/a   
1168    U.1.37.16  Stand alone authentication requirements such a...       No   
1169    U.1.37.17                    Passwords encrypted in transit?      Yes   
1170    U.1.37.18          Passwords encrypted or hashed in storage?      Yes   
1171    U.1.37.19    Passwords displayed when entered into a system?       No   
1172    U.1.37.20   User accounts associated to a unique individual?      Yes   
1173    U.1.37.21  System lock requirements after 3-5 invalid log...       No   
1174    U.1.37.22  Restricted administrative access to the manage...      Yes   
1175    U.1.37.23  Disabling unneeded hypervisor services (e.g., ...      Yes   
1176    U.1.37.24  Introspection capabilities to monitor the secu...      Yes   
1177    U.1.37.25  Introspection capabilities to monitor the secu...      Yes   
1178    U.1.37.26  Separate network VLANs for host operating syst...      Yes   
1179    U.1.37.27  Guest operating systems communicate on separat...       No   
1180    U.1.37.28  Host operating system management interface on ...      Yes   
1181    U.1.37.29  Two factor authentication for access to the ad...      Yes   
1182    U.1.37.30  Approval process before VMs can be created to ...      Yes   
1183    U.1.37.31  Migration of VMs logged including source and t...      Yes   
1184    U.1.37.32  VM's in the same 

In [9]:
#replace NaN Values

classifier.loc[:, 'misc_info'].fillna('', inplace=True)
classifier.loc[:, 'iso_desc'].fillna('', inplace=True)
classifier.loc[:, 'aup_ref'].fillna('', inplace=True)

In [10]:
# combining multiple columns to create master text


classifier.loc[:, 'textfield']   = (classifier.loc[:, 'question'] + ' ' 
                                    + classifier.loc[:, 'misc_info'] + ' ' 
                                    + classifier.loc[:, 'iso_desc'] + ' ' 
                                    + classifier.loc[:, 'aup_ref'])

In [11]:
X = classifier.loc[:, 'textfield']
y = classifier.loc[:, 'sig_class_num']

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# EDA on Question Text

In [13]:
vect = CountVectorizer()
vect.fit(X_train)
X_train_dtm = vect.transform(X_train)

In [14]:
X_train_dtm

<897x2082 sparse matrix of type '<class 'numpy.int64'>'
	with 15481 stored elements in Compressed Sparse Row format>

In [15]:
X_train_dtm.todense().shape

(897, 2082)

In [16]:
X_train_dtm.todense().sum(axis=0)

matrix([[11,  1, 19, ...,  4,  2,  3]], dtype=int64)

In [17]:
vect.get_feature_names()

['10',
 '1000s',
 '11',
 '12',
 '120',
 '140',
 '15',
 '16',
 '18',
 '180',
 '19',
 '1x',
 '2048',
 '21',
 '24',
 '24x7',
 '24x7x365',
 '25',
 '27001',
 '2731',
 '2fa',
 '30',
 '33',
 '3402',
 '35',
 '365',
 '37',
 '3rd',
 '40',
 '443',
 '60',
 '6671',
 '80',
 '800',
 '802',
 '88',
 '90',
 'ability',
 'able',
 'about',
 'above',
 'abuse',
 'accept',
 'acceptable',
 'acceptance',
 'accepted',
 'accepting',
 'access',
 'accessed',
 'accessible',
 'accessing',
 'accesssed',
 'accidental',
 'accordance',
 'according',
 'account',
 'accounting',
 'accounts',
 'accuracy',
 'acknowledge',
 'acquisition',
 'acted',
 'action',
 'actioned',
 'actions',
 'activate',
 'activating',
 'active',
 'actively',
 'activex',
 'activities',
 'activity',
 'ad',
 'addition',
 'additional',
 'additionally',
 'address',
 'addressed',
 'addresses',
 'addressing',
 'adequacy',
 'adjustment',
 'admin',
 'administration',
 'administrative',
 'administrator',
 'administrators',
 'adopted',
 'advertising',
 'advises

In [18]:
vect.vocabulary_

{'are': 155,
 'apis': 136,
 'tested': 1862,
 'for': 767,
 'security': 1659,
 'weaknesses': 2030,
 'if': 888,
 'yes': 2076,
 'does': 588,
 'this': 1875,
 'include': 911,
 'management': 1113,
 'of': 1229,
 'technical': 1843,
 'vulnerabilities': 2022,
 'is': 996,
 'scoped': 1644,
 'systems': 1825,
 'and': 125,
 'data': 470,
 'ever': 684,
 'used': 1976,
 'in': 902,
 'the': 1867,
 'test': 1861,
 'development': 539,
 'or': 1262,
 'qa': 1472,
 'environments': 660,
 'protection': 1452,
 'software': 1739,
 'program': 1436,
 'bouncycastle': 254,
 'openssl': 1248,
 'prior': 1411,
 'to': 1893,
 'device': 540,
 'on': 1240,
 'boarding': 250,
 'constituents': 396,
 'required': 1565,
 'sign': 1712,
 'legal': 1045,
 'agreement': 98,
 'which': 2040,
 'details': 525,
 'obligations': 1224,
 'rights': 1606,
 'related': 1527,
 'mobile': 1159,
 'devices': 541,
 'it': 1006,
 'policy': 1378,
 'errors': 666,
 'resulting': 1588,
 'from': 781,
 'incomplete': 915,
 'inaccurate': 903,
 'business': 267,
 'reporting'

# Train Test Classifier using Naive Bayes

In [19]:
#Create Document-term Matrices with CountVectorizer()
vect = CountVectorizer()
vect.fit(X_train)
X_train_dtm = vect.transform(X_train)
X_test_dtm = vect.transform(X_test)

#Use Naive Bayes to predict sig_class_num
nb = MultinomialNB()
nb.fit(X_train_dtm, y_train)
y_pred_class = nb.predict(X_test_dtm)

In [20]:
# calculate accuracy
metrics.accuracy_score(y_test, y_pred_class)

0.62333333333333329

In [21]:
y_test.value_counts()

1     53
3     36
2     26
4     26
5     24
6     22
7     19
9     18
8     15
12    11
15    10
11     9
14     9
10     8
13     7
17     4
16     3
Name: sig_class_num, dtype: int64

In [22]:
# Define a function that accepts a vectorizer and calculates the accuracy.

def tokenize_test(vect):
    vect.fit(X_train)
    X_train_dtm = vect.transform(X_train)
    print('Number of features: ', X_train_dtm.shape[1])
    X_test_dtm = vect.transform(X_test)
    
    nb = MultinomialNB()
    nb.fit(X_train_dtm, y_train)
    y_pred_class = nb.predict(X_test_dtm)
    print('Accuracy: ', metrics.accuracy_score(y_test, y_pred_class))

In [23]:
vect = CountVectorizer()
tokenize_test(vect)

Number of features:  2082
Accuracy:  0.623333333333


In [24]:
vect = CountVectorizer(ngram_range=(1,2))
tokenize_test(vect)

Number of features:  10439
Accuracy:  0.626666666667


In [25]:
vect = CountVectorizer(stop_words='english')

vect.get_stop_words()

frozenset({'a',
           'about',
           'above',
           'across',
           'after',
           'afterwards',
           'again',
           'against',
           'all',
           'almost',
           'alone',
           'along',
           'already',
           'also',
           'although',
           'always',
           'am',
           'among',
           'amongst',
           'amoungst',
           'amount',
           'an',
           'and',
           'another',
           'any',
           'anyhow',
           'anyone',
           'anything',
           'anyway',
           'anywhere',
           'are',
           'around',
           'as',
           'at',
           'back',
           'be',
           'became',
           'because',
           'become',
           'becomes',
           'becoming',
           'been',
           'before',
           'beforehand',
           'behind',
           'being',
           'below',
           'beside',
           'besides'

In [26]:
tokenize_test(vect)

Number of features:  1932
Accuracy:  0.63


In [27]:
vect = CountVectorizer(stop_words='english')
tokenize_test(vect)

Number of features:  1932
Accuracy:  0.63


In [28]:
vect.get_feature_names()

['10',
 '1000s',
 '11',
 '12',
 '120',
 '140',
 '15',
 '16',
 '18',
 '180',
 '19',
 '1x',
 '2048',
 '21',
 '24',
 '24x7',
 '24x7x365',
 '25',
 '27001',
 '2731',
 '2fa',
 '30',
 '33',
 '3402',
 '35',
 '365',
 '37',
 '3rd',
 '40',
 '443',
 '60',
 '6671',
 '80',
 '800',
 '802',
 '88',
 '90',
 'ability',
 'able',
 'abuse',
 'accept',
 'acceptable',
 'acceptance',
 'accepted',
 'accepting',
 'access',
 'accessed',
 'accessible',
 'accessing',
 'accesssed',
 'accidental',
 'accordance',
 'according',
 'account',
 'accounting',
 'accounts',
 'accuracy',
 'acknowledge',
 'acquisition',
 'acted',
 'action',
 'actioned',
 'actions',
 'activate',
 'activating',
 'active',
 'actively',
 'activex',
 'activities',
 'activity',
 'ad',
 'addition',
 'additional',
 'additionally',
 'address',
 'addressed',
 'addresses',
 'addressing',
 'adequacy',
 'adjustment',
 'admin',
 'administration',
 'administrative',
 'administrator',
 'administrators',
 'adopted',
 'advertising',
 'advises',
 'affect',
 'affe

In [29]:
vect = CountVectorizer(stop_words='english', ngram_range=(1,2), min_df=1)
tokenize_test(vect)

Number of features:  8258
Accuracy:  0.656666666667


# Textblob

In [30]:
question = TextBlob(classifier.loc[0, 'question'])

In [31]:
question

TextBlob("Is there a risk assessment program that has been approved by management, communicated to constituents and an owner to maintain and review the program? if yes, does it include:")

In [32]:
stemmer = SnowballStemmer('english')

In [33]:
question.words

WordList(['Is', 'there', 'a', 'risk', 'assessment', 'program', 'that', 'has', 'been', 'approved', 'by', 'management', 'communicated', 'to', 'constituents', 'and', 'an', 'owner', 'to', 'maintain', 'and', 'review', 'the', 'program', 'if', 'yes', 'does', 'it', 'include'])

In [34]:
#Stemmer is crude form
[stemmer.stem(word) for word in question.words]

['is',
 'there',
 'a',
 'risk',
 'assess',
 'program',
 'that',
 'has',
 'been',
 'approv',
 'by',
 'manag',
 'communic',
 'to',
 'constitu',
 'and',
 'an',
 'owner',
 'to',
 'maintain',
 'and',
 'review',
 'the',
 'program',
 'if',
 'yes',
 'doe',
 'it',
 'includ']

In [35]:
print([word.lemmatize(pos='v') for word in question.words])

['Is', 'there', 'a', 'risk', 'assessment', 'program', 'that', 'have', 'be', 'approve', 'by', 'management', 'communicate', 'to', 'constituents', 'and', 'an', 'owner', 'to', 'maintain', 'and', 'review', 'the', 'program', 'if', 'yes', 'do', 'it', 'include']


In [36]:
def split_into_lemmas(text):
    text = str(text).lower()
    words = TextBlob(text).words
    return (word.lemmatize(pos='v') for word in words)

In [37]:
vect = CountVectorizer(stop_words ='english', analyzer=split_into_lemmas, decode_error='replace', ngram_range=(1,1), min_df=3)
tokenize_test(vect)

Number of features:  851
Accuracy:  0.666666666667


In [38]:
classifier

question_num                                           question response  \
0             A.1  Is there a risk assessment program that has be...      Yes   
1           A.1.1  A risk assessment, conducted within the last 1...      Yes   
2           A.1.2                                   Risk Governance?      Yes   
3           A.1.3  Range of assets to include: people, processes,...      Yes   
4           A.1.4  Range of threats to include: malicious, natura...      Yes   
5           A.1.5                                      Risk scoping?      Yes   
6           A.1.6                                      Risk context?      Yes   
7           A.1.7                                Risk training plan?      Yes   
8           A.1.8                          Risk evaluation criteria?      Yes   
9           A.1.9           Risk scenarios? If yes, do they include:      Yes   
10        A.1.9.1  Events and possible threats that could impact ...      Yes   
11        A.1.9.2                                      Threat types?      Yes   
12         A.1.10  Ownership, action plan, response plan, managem...      Yes   
13            A.2  Is there a program to manage the treatment of ...      Yes   
14          A.2.1  A formal process for assigning appropriate man...      Yes   
15          A.2.2  A formal process for appropriate management kn...      Yes   
16          A.2.3  A formal process for tracking the status of ac...      Yes   
17          A.2.4        Controls identified for each material risk?      Yes   
18          A.2.5  Measures for defining, monitoring, and reporti...      Yes   
19        A.2.5.1                            Performance benchmarks?      Yes   
20        A.2.5.2                Service Level Agreement Compliance?      Yes   
21        A.2.5.3                                 Policy Compliance?      Yes   
22        A.2.5.4                             Control Effectiveness?      Yes   
23        A.2.5.5                          Vulnerability Management?      Yes   
24            A.3  Do Subcontractors have access to Scoped System...       No   
25            A.4              Is there a vendor management program?      Yes   
26          A.4.1  Does the vendor management program include an ...       No   
27        A.4.1.1  Risk rating of the issue (e.g., H/M/L, 1-5, et...       No   
28        A.4.1.2                                 Remediation plans?       No   
29        A.4.1.3                                  Remediation date?       No   
...           ...                                                ...      ...   
1167    U.1.37.15       Password change requirements at first logon?      n/a   
1168    U.1.37.16  Stand alone authentication requirements such a...       No   
1169    U.1.37.17                    Passwords encrypted in transit?      Yes   
1170    U.1.37.18          Passwords encrypted or hashed in storage?      Yes   
1171    U.1.37.19    Passwords displayed when entered into a system?       No   
1172    U.1.37.20   User accounts associated to a unique individual?      Yes   
1173    U.1.37.21  System lock requirements after 3-5 invalid log...       No   
1174    U.1.37.22  Restricted administrative access to the manage...      Yes   
1175    U.1.37.23  Disabling unneeded hypervisor services (e.g., ...      Yes   
1176    U.1.37.24  Introspection capabilities to monitor the secu...      Yes   
1177    U.1.37.25  Introspection capabilities to monitor the secu...      Yes   
1178    U.1.37.26  Separate network VLANs for host operating syst...      Yes   
1179    U.1.37.27  Guest operating systems communicate on separat...       No   
1180    U.1.37.28  Host operating system management interface on ...      Yes   
1181    U.1.37.29  Two factor authentication for access to the ad...      Yes   
1182    U.1.37.30  Approval process before VMs can be created to ...      Yes   
1183    U.1.37.31  Migration of VMs logged including source and t...      Yes   
1184    U.1.37.32  VM's in the same 

In [39]:
#help(LogisticRegression())

# Deploy model on TRC_Master Data

In [40]:
trc_file = './data/trc_master_train_data.csv'

In [41]:
trc_test_data = pd.read_csv(trc_file, encoding='latin1')

In [42]:
trc_test_data

Tag  \
0                                        access control   
1                                        access control   
2                                        access control   
3                                 access control policy   
4                                      Access Control,    
5                             access control, accounts,   
6           access control, authorization, system admin   
7     access control, employee management system, le...   
8                       access control, password policy   
9           access control, password policy, encryption   
10                Access Control, Review, access review   
11                          access control, review, log   
12                                       access removal   
13    accounting system, upload, holdings, import, e...   
14                                              ActiveX   
15                           ActiveX, HTML, Flash, Java   
16                                      addins, add-ins   
17    admin access, admin rights, administrator acce...   
18                                                alert   
19                                 alert, vulnerability   
20                                             alerting   
21     Antivirus, anti-virus, anti-malware, antimalware   
22     Antivirus, anti-virus, anti-malware, antimalware   
23     Antivirus, anti-virus, anti-malware, antimalware   
24     Antivirus, anti-virus, anti-malware, antimalware   
25     Antivirus, anti-virus, anti-malware, antimalware   
26     Antivirus, anti-virus, anti-malware, antimalware   
27     Antivirus, anti-virus, anti-malware, antimalware   
28     Antivirus, anti-virus, anti-malware, antimalware   
29     Antivirus, anti-virus, anti-malware, antimalware   
...                                                 ...   
1039                                                NaN   
1040                                                NaN   
1041                                                NaN   
1042                                                NaN   
1043                                                NaN   
1044                                                NaN   
1045                                                NaN   
1046                                                NaN   
1047                                                NaN   
1048                                                NaN   
1049                                                NaN   
1050                                                NaN   
1051                                                NaN   
1052                                                NaN   
1053                                                NaN   
1054                                                NaN   
1055                                                NaN   
1056                                                NaN   
1057                                                NaN   
1058                                                NaN   
1059                                                NaN   
1060                                                NaN   
1061                                                NaN   
1062                                                NaN   
1063                                                NaN   
1064                                                NaN   
1065                                                NaN   
1066                                                NaN   
1067                                                NaN   
1068                                                NaN   

                                               Question  \
0     Are all system login screens configured to dis...   
1     Does your company disable default privileged a...   
2                                                   NaN   
3        copy of your Access Control Policy and process   
4     Please explain who initiates and approves the ...   
5     Is there a dedicated team tasked 

In [43]:
# Fill in NaN data with blanks
trc_test_data.loc[:, 'Tag'].fillna('', inplace=True)
trc_test_data.loc[:, 'Question'].fillna('', inplace=True)
trc_test_data.loc[:, 'Answer'].fillna('', inplace=True)

In [44]:
# Concatenate text columns
trc_test_data.loc[:, 'text_concat'] = (trc_test_data.loc[:, 'Tag'] + ' ' 
                                       + trc_test_data.loc[:, 'Question'] + ' ' 
                                       + trc_test_data.loc[:, 'Answer'])

In [45]:
trc_test_data

Tag  \
0                                        access control   
1                                        access control   
2                                        access control   
3                                 access control policy   
4                                      Access Control,    
5                             access control, accounts,   
6           access control, authorization, system admin   
7     access control, employee management system, le...   
8                       access control, password policy   
9           access control, password policy, encryption   
10                Access Control, Review, access review   
11                          access control, review, log   
12                                       access removal   
13    accounting system, upload, holdings, import, e...   
14                                              ActiveX   
15                           ActiveX, HTML, Flash, Java   
16                                      addins, add-ins   
17    admin access, admin rights, administrator acce...   
18                                                alert   
19                                 alert, vulnerability   
20                                             alerting   
21     Antivirus, anti-virus, anti-malware, antimalware   
22     Antivirus, anti-virus, anti-malware, antimalware   
23     Antivirus, anti-virus, anti-malware, antimalware   
24     Antivirus, anti-virus, anti-malware, antimalware   
25     Antivirus, anti-virus, anti-malware, antimalware   
26     Antivirus, anti-virus, anti-malware, antimalware   
27     Antivirus, anti-virus, anti-malware, antimalware   
28     Antivirus, anti-virus, anti-malware, antimalware   
29     Antivirus, anti-virus, anti-malware, antimalware   
...                                                 ...   
1039                                                      
1040                                                      
1041                                                      
1042                                                      
1043                                                      
1044                                                      
1045                                                      
1046                                                      
1047                                                      
1048                                                      
1049                                                      
1050                                                      
1051                                                      
1052                                                      
1053                                                      
1054                                                      
1055                                                      
1056                                                      
1057                                                      
1058                                                      
1059                                                      
1060                                                      
1061                                                      
1062                                                      
1063                                                      
1064                                                      
1065                                                      
1066                                                      
1067                                                      
1068                                                      

                                               Question  \
0     Are all system login screens configured to dis...   
1     Does your company disable default privileged a...   
2                                                         
3        copy of your Access Control Policy and process   
4     Please explain who initiates and approves the ...   
5     Is there a dedicated team tasked 

In [46]:
X_master_test = trc_test_data.loc[:, 'text_concat']

In [47]:


#Create Document-term Matrices with CountVectorizer()
vect = CountVectorizer(stop_words ='english', analyzer=split_into_lemmas, decode_error='replace', ngram_range=(1,1), min_df=3)
vect.fit(X_train)
X_train_dtm = vect.transform(X_train)
# X_test_dtm = vect.transform(X_test)
X_master_test_dtm = vect.transform(X_master_test)

#Use Naive Bayes to predict sig_class_num
nb = MultinomialNB()
nb.fit(X_train_dtm, y_train)
y_pred_class = nb.predict(X_master_test_dtm)

In [48]:
# Convert y_pred_class results into a dataframe
class_pred = pd.DataFrame(y_pred_class)

In [49]:
trc_with_class_pred = pd.concat([trc_test_data, class_pred], axis=1)

In [50]:
trc_with_class_pred

Tag  \
0                                        access control   
1                                        access control   
2                                        access control   
3                                 access control policy   
4                                      Access Control,    
5                             access control, accounts,   
6           access control, authorization, system admin   
7     access control, employee management system, le...   
8                       access control, password policy   
9           access control, password policy, encryption   
10                Access Control, Review, access review   
11                          access control, review, log   
12                                       access removal   
13    accounting system, upload, holdings, import, e...   
14                                              ActiveX   
15                           ActiveX, HTML, Flash, Java   
16                                      addins, add-ins   
17    admin access, admin rights, administrator acce...   
18                                                alert   
19                                 alert, vulnerability   
20                                             alerting   
21     Antivirus, anti-virus, anti-malware, antimalware   
22     Antivirus, anti-virus, anti-malware, antimalware   
23     Antivirus, anti-virus, anti-malware, antimalware   
24     Antivirus, anti-virus, anti-malware, antimalware   
25     Antivirus, anti-virus, anti-malware, antimalware   
26     Antivirus, anti-virus, anti-malware, antimalware   
27     Antivirus, anti-virus, anti-malware, antimalware   
28     Antivirus, anti-virus, anti-malware, antimalware   
29     Antivirus, anti-virus, anti-malware, antimalware   
...                                                 ...   
1039                                                      
1040                                                      
1041                                                      
1042                                                      
1043                                                      
1044                                                      
1045                                                      
1046                                                      
1047                                                      
1048                                                      
1049                                                      
1050                                                      
1051                                                      
1052                                                      
1053                                                      
1054                                                      
1055                                                      
1056                                                      
1057                                                      
1058                                                      
1059                                                      
1060                                                      
1061                                                      
1062                                                      
1063                                                      
1064                                                      
1065                                                      
1066                                                      
1067                                                      
1068                                                      

                                               Question  \
0     Are all system login screens configured to dis...   
1     Does your company disable default privileged a...   
2                                                         
3        copy of your Access Control Policy and process   
4     Please explain who initiates and approves the ...   
5     Is there a dedicated team tasked 

In [51]:
#Naming the last columns to class_num

trc_with_class_pred.columns = ['Tag','Question','Answer','Updated date','Answer provided by','Expiration Date','text_concat','pred_class_num']

In [52]:
trc_with_class_pred.loc[:, 'pred_class'] = trc_with_class_pred.loc[:, 'pred_class_num'].map({
    1:'Application Security',
    2:'Operations Management',
    3:'Business Resiliency',
    4:'Asset and Information Management',
    5:'Server Security',
    6:'Physical and Environment',
    7:'End User Device Security',
    8:'Access Control',
    9:'Network Security',
    10:'Incident Event & Communication Management',
    11:'Risk Assessment',
    12:'Security Policy',
    13:'Threat Management',
    14:'Compliance',
    15:'Human Resource Security',
    16:'Privacy',
    17:'Organizational Security'
})

In [53]:
trc_with_class_pred

Tag  \
0                                        access control   
1                                        access control   
2                                        access control   
3                                 access control policy   
4                                      Access Control,    
5                             access control, accounts,   
6           access control, authorization, system admin   
7     access control, employee management system, le...   
8                       access control, password policy   
9           access control, password policy, encryption   
10                Access Control, Review, access review   
11                          access control, review, log   
12                                       access removal   
13    accounting system, upload, holdings, import, e...   
14                                              ActiveX   
15                           ActiveX, HTML, Flash, Java   
16                                      addins, add-ins   
17    admin access, admin rights, administrator acce...   
18                                                alert   
19                                 alert, vulnerability   
20                                             alerting   
21     Antivirus, anti-virus, anti-malware, antimalware   
22     Antivirus, anti-virus, anti-malware, antimalware   
23     Antivirus, anti-virus, anti-malware, antimalware   
24     Antivirus, anti-virus, anti-malware, antimalware   
25     Antivirus, anti-virus, anti-malware, antimalware   
26     Antivirus, anti-virus, anti-malware, antimalware   
27     Antivirus, anti-virus, anti-malware, antimalware   
28     Antivirus, anti-virus, anti-malware, antimalware   
29     Antivirus, anti-virus, anti-malware, antimalware   
...                                                 ...   
1039                                                      
1040                                                      
1041                                                      
1042                                                      
1043                                                      
1044                                                      
1045                                                      
1046                                                      
1047                                                      
1048                                                      
1049                                                      
1050                                                      
1051                                                      
1052                                                      
1053                                                      
1054                                                      
1055                                                      
1056                                                      
1057                                                      
1058                                                      
1059                                                      
1060                                                      
1061                                                      
1062                                                      
1063                                                      
1064                                                      
1065                                                      
1066                                                      
1067                                                      
1068                                                      

                                               Question  \
0     Are all system login screens configured to dis...   
1     Does your company disable default privileged a...   
2                                                         
3        copy of your Access Control Policy and process   
4     Please explain who initiates and approves the ...   
5     Is there a dedicated team tasked 

In [55]:
trc_with_class_pred.to_csv('./data/trc_pred_class.csv')